标准库里的所有映射类型都是利用 dict 来实现的，因此它们有个共同的限制，即只有可散列的数据类型才能用作这些映射里的键（只有键有
这个要求，值并不需要是可散列的数据类型）。

### 什么是可散列的数据类型

如果一个对象是可散列的，那么在这个对象的生命周期中，它
的散列值是不变的，而且这个对象需要实现 __hash__() 方
法。另外可散列对象还要有 __qe__() 方法，这样才能跟其他
键做比较。如果两个可散列对象是相等的，那么它们的散列值
一定是一样的……

原子不可变数据类型（str、bytes 和数值类型）都是可散列类
型，frozenset 也是可散列的，因为根据其定义，frozenset 里
只能容纳可散列类型。元组的话，只有当一个元组包含的所有元素
都是可散列类型的情况下，它才是可散列的。

In [1]:
tt = (1, 2, (30, 40))
hash(tt)

8027212646858338501

In [2]:
tl = (1, 2, [30, 40])
hash(tl)

TypeError: unhashable type: 'list'

In [3]:
tf = (1, 2, frozenset([30, 40]))
hash(tf)

-4118419923444501110

Python 词汇表
（https://docs.python.org/3/glossary.html#term-hashable）里还在
说“Python 里所有的不可变类型都是可散列的”。这个说法其实是不
准确的，比如虽然元组本身是不可变序列，它里面的元素可能是其
他可变类型的引用.一般来讲用户自定义的类型的对象都是可散列的，散列值就是它们
的 id() 函数的返回值，所以所有这些对象在比较的时候都是不相
等的。如果一个对象实现了 __eq__ 方法，并且在方法中用到了这
个对象的内部状态的话，那么只有当所有这些内部状态都是不可变
的情况下，这个对象才是可散列的。

### 字典推导

利用字典推导可以把一个装满元组的列表变成两
个不同的字典

In [4]:
DIAL_CODES = [ 
 (86, 'China'),
 (91, 'India'),
 (1, 'United States'),
 (62, 'Indonesia'),
 (55, 'Brazil'),
 (92, 'Pakistan'),
 (880, 'Bangladesh'),
 (234, 'Nigeria'),
 (7, 'Russia'),
(81, 'Japan'),
 ]

country_code = {country: code for code, country in DIAL_CODES}
country_code

{'Bangladesh': 880,
 'Brazil': 55,
 'China': 86,
 'India': 91,
 'Indonesia': 62,
 'Japan': 81,
 'Nigeria': 234,
 'Pakistan': 92,
 'Russia': 7,
 'United States': 1}

In [5]:
{code: country.upper() for country, code in country_code.items() if code < 66}

{1: 'UNITED STATES', 7: 'RUSSIA', 55: 'BRAZIL', 62: 'INDONESIA'}

### 用setdefault处理找不到的键

当字典 d[k] 不能找到正确的键的时候，Python 会抛出异常，这个行为
符合 Python 所信奉的“快速失败”哲学。也许每个 Python 程序员都知道
可以用 d.get(k, default) 来代替 d[k]，给找不到的键一个默认的
返回值（这比处理 KeyError 要方便不少）。但是要更新某个键对应的值的时候，不管使用 __getitem__ 还是 get 都会不自然，而且效率
低。

获取单词的出现情况列表，如果单词不存在，把单词和一个空列表
放进映射，然后返回这个空列表，这样就能在不进行第二次查找的情况
下更新列表了

#这样写
my_dict.setdefault(key, []).append(new_value)
#跟这样写是一样的
if key not in my_dict:
    my_dict[key] = []
my_dict[key].append(new_value)

二者的效果是一样的，只不过后者至少要进行两次键查询——如果键不
存在的话，就是三次，用 setdefault 只需要一次就可以完成整个操
作。

映射的弹性键查询        
有时候为了方便起见，就算某个键在映射里不存在，我们也希望在通过
这个键读取值的时候能得到一个默认值。有两个途径能帮我们达到这个
目的，一个是通过 defaultdict 这个类型而不是普通的 dict，另一个
是给自己定义一个 dict 的子类，然后在子类中实现 __missing__ 方
法。下面将介绍这两种方法。

### defaultdict：处理找不到的键的一个选择     
在实例化一个 defaultdict 的时候，需要给构造方法提供
一个可调用对象，这个可调用对象会在 __getitem__ 碰到找不到的键
的时候被调用，让 __getitem__ 返回某种默认值。     
比如，我们新建了这样一个字典：dd = defaultdict(list)，如果键
'new-key' 在 dd 中还不存在的话，表达式 dd['new-key'] 会按照以
下的步骤来行事。    
(1) 调用 list() 来建立一个新列表。       
(2) 把这个新列表作为值，'new-key' 作为它的键，放到 dd 中。    
(3) 返回这个列表的引用  

所有这一切背后的功臣其实是特殊方法 __missing__。它会在
defaultdict 遇到找不到的键的时候调用 default_factory

所有这一切背后的功臣其实是特殊方法 __missing__。它会在
defaultdict 遇到找不到的键的时候调用 default_factory，特殊方法__missing__
所有的映射类型在处理找不到的键的时候，都会牵扯到 __missing__
方法。这也是这个方法称作“missing”的原因。虽然基类 dict 并没有定
义这个方法，但是 dict 是知道有这么个东西存在的。也就是说，如果
有一个类继承了 dict，然后这个继承类提供了 __missing__ 方法，那
么在 __getitem__ 碰到找不到的键的时候，Python 就会自动调用它，
而不是抛出一个 KeyError 异常。

如果要自定义一个映射类型，更合适的策略其实是继承
collections.UserDict 类（示例 3-8 就是如此）。这里我们从
dict 继承，只是为了演示 __missing__ 是如何被
dict.__getitem__ 调用的

In [ ]:
class StrKeyDict0(dict): ➊
    def __missing__(self, key):
        if isinstance(key, str): ➋
            raise KeyError(key)
        return self[str(key)] ➌
    def get(self, key, default=None):
        try:
            return self[key] ➍
        except KeyError:
            return default ➎
    def __contains__(self, key):
        return key in self.keys() or str(key) in self.keys() ➏

❶ StrKeyDict0 继承了 dict。  
❷ 如果找不到的键本身就是字符串，那就抛出 KeyError 异常。  
❸ 如果找不到的键不是字符串，那么把它转换成字符串再进行查找。  
❹ get 方法把查找工作用 self[key] 的形式委托给 __getitem__，这
样在宣布查找失败之前，还能通过 __missing__ 再给某个键一个机
会。  
❺ 如果抛出 KeyError，那么说明 __missing__ 也失败了，于是返回
default。    
❻ 先按照传入键的原本的值来查找（我们的映射类型中可能含有非字
符串的键），如果没找到，再用 str() 方法把键转换成字符串再查找
一次  

### 字典的变种

### collections.OrderedDict     
这个类型在添加键的时候会保持顺序，因此键的迭代次序总是一致
的。OrderedDict 的 popitem 方法默认删除并返回的是字典里的最后
一个元素，但是如果像 my_odict.popitem(last=False) 这样调用
它，那么它删除并返回第一个被添加进去的元素。  

### collections.ChainMap
该类型可以容纳数个不同的映射对象，然后在进行键查找操作的时
候，这些对象会被当作一个整体被逐个查找，直到键被找到为止。这个
功能在给有嵌套作用域的语言做解释器的时候很有用，可以用一个映射
对象来代表一个作用域的上下文。在 collections 文档介绍 ChainMap
对象的那一部分
（https://docs.python.org/3/library/collections.html#collections.ChainMap）
里有一些具体的使用示例，

### collections.Counter    
这个映射类型会给键准备一个整数计数器。每次更新一个键的时候
都会增加这个计数器。所以这个类型可以用来给可散列表对象计数，或
者是当成多重集来用——多重集合就是集合里的元素可以出现不止一
次。Counter 实现了 + 和 - 运算符用来合并记录，还有像
most_common([n]) 这类很有用的方法。most_common([n]) 会按照次
序返回映射里最常见的 n 个键和它们的计数，详情参阅文档
（https://docs.python.org/3/library/collections.html#collections.Counter）。

In [8]:
import collections
ct = collections.Counter('abracadabra')
ct

Counter({'a': 5, 'b': 2, 'c': 1, 'd': 1, 'r': 2})

In [9]:
ct.update('aaaaazzz')
ct

Counter({'a': 10, 'b': 2, 'c': 1, 'd': 1, 'r': 2, 'z': 3})

In [10]:
ct.most_common(2)

[('a', 10), ('z', 3)]

### colllections.UserDict
这个类其实就是把标准 dict 用纯 Python 又实现了一遍。
跟 OrderedDict、ChainMap 和 Counter 这些开箱即用的类型不
同，UserDict 是让用户继承写子类的。

### 子类化UserDict
就创造自定义映射类型来说，以 UserDict 为基类，总比以普通的
dict 为基类要来得方便。
这体现在，我们能够改进示例 3-7 中定义的 StrKeyDict0 类，使得所
有的键都存储为字符串类型。
而更倾向于从 UserDict 而不是从 dict 继承的主要原因是，后者有时
会在某些方法的实现上走一些捷径，导致我们不得不在它的子类中重写
这些方法，但是 UserDict 就不会带来这些问题

另外一个值得注意的地方是，UserDict 并不是 dict 的子类，但是
UserDict 有一个叫作 data 的属性，是 dict 的实例，这个属性实际上
是 UserDict 最终存储数据的地方。这样做的好处是，比起示例 3-
7，UserDict 的子类就能在实现 __setitem__ 的时候避免不必要的递
归，也可以让 __contains__ 里的代码更简洁。
多亏了 UserDict，示例 3-8 里的 StrKeyDict 的代码比示例 3-7 里的
StrKeyDict0 要短一些，功能却更完善：它不但把所有的键都以字符
串的形式存储，还能处理一些创建或者更新实例时包含非字符串类型的
键这类意外情况

In [ ]:
import collections
class StrKeyDict(collections.UserDict): ➊
def __missing__(self, key): ➋
if isinstance(key, str):
raise KeyError(key)
return self[str(key)]
def __contains__(self, key):
return str(key) in self.data ➌
def __setitem__(self, key, item):
self.data[str(key)] = item ➍

❶ StrKeyDict 是对 UserDict 的扩展。   
❷ __missing__ 跟示例 3-7 里的一模一样。   
❸ __contains__ 则更简洁些。这里可以放心假设所有已经存储的键都
是字符串。因此，只要在 self.data 上查询就好了，并不需要像
StrKeyDict0 那样去麻烦 self.keys()。   
❹ __setitem__ 会把所有的键都转换成字符串。由于把具体的实现委
托给了 self.data 属性，这个方法写起来也不难  

### 不可变映射类型

标准库里所有的映射类型都是可变的，但有时候你会有这样的需求，比
如不能让用户错误地修改某个映射。3.4.2 节提到过 Pingo.io，它里面就
有个现成的例子。Pingo.io 里有个映射的名字叫作 board.pins，里面
的数据是 GPIO 物理针脚的信息，我们当然不希望用户一个疏忽就把这
些信息给改了。因为硬件方面的东西是不会受软件影响的，所以如果把
这个映射里的信息改了，就跟物理上的元件对不上号了。
从 Python 3.3 开始，types 模块中引入了一个封装类名叫
MappingProxyType。如果给这个类一个映射，它会返回一个只读的映
射视图。虽然是个只读视图，但是它是动态的。这意味着如果对原映射
做出了改动，我们通过这个视图可以观察到，但是无法通过这个视图对
原映射做出修改。示例 3-9 简短地对这个类的用法做了个演示。

In [11]:
from types import MappingProxyType
d = {1:'A'}
d_proxy = MappingProxyType(d)

In [12]:
d_proxy[1]

'A'

In [13]:
d_proxy[2] = 'x'

TypeError: 'mappingproxy' object does not support item assignment

In [14]:
d[2] = 'B'
d_proxy[2]

'B'